In [4]:

import numpy as np
import os
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm, Normalize
import time
import pickle

from concurrent.futures import ProcessPoolExecutor, as_completed

import discretize
# from simpeg import dask
from simpeg.utils import mkvc, plot_1d_layer_model
from simpeg import (
    maps,
    Data,
    data_misfit,
    inverse_problem,
    regularization,
    optimization,
    directives,
    inversion,
    utils,
)
from simpeg.electromagnetics import time_domain as tdem
from simpeg.utils.solver_utils import get_default_solver

from simpeg.meta import MultiprocessingMetaSimulation, MetaSimulation


Solver = get_default_solver()


directory = "../synthetic-data-10m"
files = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f)) and f.split(".")[-1]=="txt"]

files.remove("rx_locs.txt")
rx_locs = np.loadtxt(f"{directory}/rx_locs.txt")
# rx_locs[:, 1] =

files.remove("rx_times.txt")
rx_times = np.loadtxt(f"{directory}/rx_times.txt")


In [5]:
dobs_dict = {}

for f in files:
    key = f.split(".")[0]
    dobs_dict[key] = np.loadtxt(f"{directory}/{f}")
    
data_dict = {}
for key, value in dobs_dict.items():
    source_list = []
    for i in range(rx_locs.shape[0]):
        rx = tdem.receivers.PointMagneticFluxTimeDerivative(rx_locs[i, :], rx_times, orientation="z")
        src = tdem.sources.CircularLoop(
            receiver_list=[rx], location=rx_locs[i, :], orientation="z", radius=10,
            waveform=tdem.sources.StepOffWaveform()
        )
        source_list.append(src)

    full_survey = tdem.Survey(source_list)

    data_dict[key] = Data(survey=full_survey, dobs=value)



In [6]:
n_times_invert = 20
data_dict_invert = {}
    
for key, value in dobs_dict.items():

    source_list = []

    for i in range(rx_locs.shape[0]):
        rx = tdem.receivers.PointMagneticFluxTimeDerivative(rx_locs[i, :], rx_times[:n_times_invert], orientation="z")
        src = tdem.sources.CircularLoop(
            receiver_list=[rx], location=rx_locs[i, :], orientation="z", radius=10,
            waveform=tdem.sources.StepOffWaveform()
        )
        source_list.append(src)

    survey = tdem.Survey(source_list)

    data_dict_invert[key] = Data(
        survey=survey,
        dobs=(value.reshape(rx_locs.shape[0], len(rx_times))[:, :n_times_invert]).flatten(),
        relative_error=0.1,
        noise_floor=1e-11
    )

In [7]:
src = source_list[0]

In [8]:
rx = src.receiver_list[0]

In [10]:
rx.locations[0, 1]

np.float64(0.0)

In [11]:
rx.times

array([0.0001    , 0.00011836, 0.00014008, 0.0001658 , 0.00019624,
       0.00023226, 0.0002749 , 0.00032536, 0.00038509, 0.00045579,
       0.00053946, 0.00063849, 0.0007557 , 0.00089443, 0.00105862,
       0.00125296, 0.00148297, 0.00175521, 0.00207742, 0.00245878])

In [12]:
rx_times

array([0.0001    , 0.00011836, 0.00014008, 0.0001658 , 0.00019624,
       0.00023226, 0.0002749 , 0.00032536, 0.00038509, 0.00045579,
       0.00053946, 0.00063849, 0.0007557 , 0.00089443, 0.00105862,
       0.00125296, 0.00148297, 0.00175521, 0.00207742, 0.00245878,
       0.00291015, 0.00344438, 0.00407668, 0.00482506, 0.00571082,
       0.00675918, 0.008     ])

In [29]:
with open("survey.txt", "w") as f:
    f.write(f"N_TRX {len(full_survey.source_list)}\n\n")

    for src in source_list: 
        f.write(f"TRX_LOOP\n")
        f.write(f"{src.location[0]} {src.location[1]} {src.location[2]} {src.radius} 0 0\n\n")

        f.write(f"N_RECV 1\n")
        f.write(f"N_TIME {n_times_invert}\n")
        for t in rx_times[:n_times_invert]:
            rx = src.receiver_list[0]
            f.write(f"{rx.locations[0, 0]} {rx.locations[0, 1]} {rx.locations[0, 2]} {t}\n")
        f.write("\n")
            


        

In [30]:
data_invert = data_dict_invert["target_0"]
survey = data_invert.survey

In [31]:
src = survey.source_list[0]
rx = src.receiver_list[0]
data_invert[src, rx]

array([-6.55434462e-09, -4.63707831e-09, -3.32808904e-09, -2.36915869e-09,
       -1.66094531e-09, -1.15095849e-09, -7.89255554e-10, -5.34561600e-10,
       -3.57517766e-10, -2.39363793e-10, -1.58666024e-10, -1.03423493e-10,
       -6.59916590e-11, -4.14325693e-11, -2.61372424e-11, -1.66247136e-11,
       -1.03789112e-11, -6.38048911e-12, -3.89142202e-12, -2.36408146e-12])

In [32]:
relative_error = 0.1
noise_floor = 1e-11

In [34]:
with open("dobs.txt", "w") as f:
    f.write(f"IGNORE -9.9999000e+004")
    f.write(f"N_TRX {len(full_survey.source_list)}\n\n")

    for src in survey.source_list: 
        f.write(f"TRX_LOOP\n")
        f.write(f"{src.location[0]} {src.location[1]} {src.location[2]} {src.radius} 0 0\n\n")

        f.write(f"N_RECV 1\n")
        f.write(f"N_TIME {n_times_invert}\n")
        for i, t in enumerate(rx_times[:n_times_invert]):
            rx = src.receiver_list[0]
            datum = -data_invert[src, rx][i]
            std = np.abs(datum) * relative_error + noise_floor
            f.write(
                f"{rx.locations[0, 0]} {rx.locations[0, 1]} {rx.locations[0, 2]} {t} "+
                f"-9.9999000e+004 -9.9999000e+004 -9.9999000e+004 -9.9999000e+004 -9.9999000e+004 -9.9999000e+004 " + # E data 
                f"-9.9999000e+004 " + # H data
                f"-9.9999000e+004 -9.9999000e+004 -9.9999000e+004 -9.9999000e+004 {datum} {std}\n"
            )
            
        f.write("\n")
            


        

In [35]:
time_steps = [
    (1e-6, 30),
    (3e-6, 30),
    (1e-5, 30), (3e-5, 20), (1e-4, 20), #(3e-4, 20)
]

In [43]:
for dt, n in time_steps: 
    print(dt, n)

1e-06 30
3e-06 30
1e-05 30
3e-05 20
0.0001 20


In [45]:
with open("waveform.txt", "w") as f:
    f.write(f"{timestep}\t 1\n")
    for dt, n in time_steps: 
        f.write(f"{dt}\t 0\t {n}\n")